# Problem
Some S2 tiles create an error when STAC metadata is created using the terrabyte ingestion lib based on stactools sentinel-2. It is related to crossing the antimeridian. Here's a documentation of the problem.

## Inputs

In [1]:
import os
scene_path = "/dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE"
scene_id = os.path.splitext(os.path.basename(scene_path))[0]
scene_id

'S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056'

## Using terrabyte ingestion lib
```
metadata_error: Error during creating metadata for /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE: Area of geometry is 64800.0, which is too large to be correct.
```

In [2]:
from terrabyte.ingestion.datasets import sentinel

In [3]:
stac_file = sentinel.sentinel_metadata(scene_path, scene_id)

executing sentinel_metadata for /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE


/dss/dsshome1/06/di38laz/micromamba/envs/s2_dateline/lib/python3.13/site-packages/antimeridian/_implementation.py:441: FixWindingWarning: The exterior ring of this shape is wound clockwise. Since this is a common error in real-world geometries, this package is reversing the exterior coordinates of the input shape before running its algorithm. If you know that your input shape is correct (i.e. if your data encompasses both poles), pass `fix_winding=False`.
  FixWindingWarning.warn()


Exception: metadata_error: Error during creating metadata for /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE: Area of geometry is 64800.0, which is too large to be correct.

## Using stactools-sentinel
Based on https://github.com/DLR-terrabyte/stactools-sentinel2

```
Exception: Area of geometry is 64800.0, which is too large to be correct.
```

In [4]:
from stactools.sentinel2.stac import create_item

In [5]:
stac = create_item(scene_path)

/dss/dsshome1/06/di38laz/micromamba/envs/s2_dateline/lib/python3.13/site-packages/antimeridian/_implementation.py:441: FixWindingWarning: The exterior ring of this shape is wound clockwise. Since this is a common error in real-world geometries, this package is reversing the exterior coordinates of the input shape before running its algorithm. If you know that your input shape is correct (i.e. if your data encompasses both poles), pass `fix_winding=False`.
  FixWindingWarning.warn()


Exception: Area of geometry is 64800.0, which is too large to be correct.

## Check versions of used packages
Theoretically the issue should be solved in the stactools sentinel2 package. But the case we experience still occurs. Also with the newest version of the package.

**Issue should be solved**: https://github.com/stactools-packages/sentinel2/issues/149

- Version on conda too old: on conda-forge 27.03.2025
`+ stactools                                0.5.3  pyhd8ed1ab_0          conda-forge/noarch         47kB`
- But: DLR Fork uses **new version** including the fix see here: https://github.com/DLR-terrabyte/stactools-sentinel2/
- It still doesn't fix that problem.
- Problem is this code part: https://github.com/DLR-terrabyte/stactools-sentinel2/blob/main/src/stactools/sentinel2/stac.py#L137


## Check against CDSE
- https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056
- https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056?.language=de

In [6]:
#bbox_cdse = (-179.99884, 69.247239, 180, 70.277197)
#gdf_cdse = gpd.GeoDataFrame(
#    {"geometry": [box(*bbox_cdse)]}, 
#    crs="EPSG:4326"  # Use the CRS of the dataset
#)
#print(gdf_cdse.total_bounds)
#gdf_cdse.explore()

# Test: Solution based on inventory mapping solution
For the inventory maps we had a similar problem to solve: 
https://gitlab.dlr.de/terrabyte/data-management/inventory/terrabyte-inventory-mapping/-/blob/main/dateline_corrections/dateline_corrections_execute.ipynb?ref_type=heads

## Micromamba Environment
`micromamba create -n s2_dateline -c conda-forge gdal python geopandas xarray pandas pyproj numpy folium shapely pip jupyterlab`

`pip install git+https://github.com/DLR-terrabyte/stactools-sentinel2.git`

`pip install git+https://gitlab.dlr.de/terrabyte/data-management/ingestion/terrabyte-ingestion-lib.git` 

In [7]:
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime
from pytz import UTC
from random import randint
from pyproj import CRS
from shapely import wkt
from shapely.geometry import LineString, MultiPolygon, Polygon
from shapely.ops import split
from math import sqrt
from numpy import sqrt

## Functions
These functions are needed to solve the dateline crossing geomertries:
- `crosses_dateline()` -> Check if dateline is crossed.
- `correct_dateline_crossing()` -> Fix the crossing by keeping the larger portion of the geometry intact and cutting at 180.

In [8]:
def crosses_dateline(geometry):
    """
    Check if a Polygon crosses the dateline (+/-180 degrees) by having both positive and negative longitudes,
    and those longitudes are near 180 degrees. This function excludes crossings near 0 degrees.
    
    Parameters:
    geometry (shapely.geometry.Polygon): A Polygon geometry to check.
    
    Returns:
    Boolean: True if the polygon crosses the dateline (±180 longitude), otherwise False.
    """
    if isinstance(geometry, Polygon):
        # Get the longitudes of the exterior ring
        longitudes = [coord[0] for coord in geometry.exterior.coords]
        
        # Check if there are both positive and negative longitudes
        has_positive = any(lon > 0 for lon in longitudes)
        has_negative = any(lon < 0 for lon in longitudes)

        # Check if the longitudes are near the dateline (close to ±180 degrees)
        near_dateline = any(abs(lon) > 170 for lon in longitudes)

        # The polygon crosses the dateline if it has both positive and negative longitudes,
        # and at least one longitude is close to 180 degrees (±170 or more).
        return has_positive and has_negative and near_dateline
    else:
        return False  # Return False if the geometry is not a Polygon
# Example use: 
# gdf_crossing = gdf[gdf['geometry'].apply(crosses_dateline)]

In [9]:
def correct_dateline_crossing(geometry):
    """Corrects polygons that cross the dateline. There are 3 possible cases: 
    Case 1: If all longitudes have the same sign, skip the geometry. Return original.
    Case 2: If there's exactly one positive or one negative longitude. Replace the sign.
    Case 3: If there are two positive and two negative longitudes. Calculate distance to 180. Then whichever distance is smaller replace those longitudes with 180 and flip their sign.
    
    Parameters:
    geometry (shapely.geometry.Polygon): A Polygon that needs to be corrected identified by crosses_dateline().

    Returns:
    geometry (shapely.geometry.Polygon): The corrected Polygon, not crossing the dateline anymore.
    
    """
    # Ensure the geometry is a Polygon
    if not isinstance(geometry, Polygon):
        return geometry  
    
    # Extract the longitudes and latitudes from the geometry
    coords = list(geometry.exterior.coords)
    lons = [lon for lon, lat in coords]
    lats = [lat for lon, lat in coords]

    # Ensure the polygon's first and last coordinates are the same
    if coords[0] != coords[-1]:
        raise ValueError("Polygon is not closed properly; first and last coordinates must be identical.")

    # Remove the last coordinate (duplicate of the first)
    lons.pop()
    lats.pop()

    # Check the signs of all longitudes
    pos_lons = [lon for lon in lons if lon > 0]
    neg_lons = [lon for lon in lons if lon < 0]

    # Case 1: If all longitudes have the same sign, skip the geometry
    if len(pos_lons) == 0 or len(neg_lons) == 0:
        return geometry
    
    # Case 2: If there's exactly one positive or one negative longitude
    elif len(pos_lons) == 1:
        # Only one positive longitude, replace it with 180
        new_lons = [-180 if lon > 0 else lon for lon in lons]
    
    elif len(neg_lons) == 1:
        # Only one negative longitude, replace it with -180
        new_lons = [180 if lon < 0 else lon for lon in lons]
    
    # Case 3: If there are two positive and two negative longitudes
    elif len(pos_lons) == 2 and len(neg_lons) == 2:
        # Calculate distances to 180 for positive and negative longitudes
        pos_distances = [180 - lon for lon in pos_lons]
        neg_distances = [abs(-180 + lon) for lon in neg_lons]

        if sum(pos_distances) < sum(neg_distances):
            # Replace positive longitudes with 180 and flip their sign
            new_lons = [180 if lon > 0 else lon for lon in lons]
            new_lons = [-180 if lon == 180 else lon for lon in new_lons]
        else:
            # Replace negative longitudes with -180 and flip their sign
            new_lons = [-180 if lon < 0 else lon for lon in lons]
            new_lons = [180 if lon == -180 else lon for lon in new_lons]
    else:
        # Other cases, return the geometry unchanged
        return geometry

    # Close the polygon loop by adding the first coordinate at the end
    new_coords = [(lon, lat) for lon, lat in zip(new_lons, lats)]
    new_coords.append(new_coords[0])  # Ensure the polygon is closed

    return Polygon(new_coords)

# Example usage:
# Assuming you have a GeoDataFrame 'gdf':
# gdf['geometry'] = gdf['geometry'].apply(correct_dateline_crossing)

## Investigate one example in detail
### Get metadata 

In [10]:
from osgeo import gdal
import xarray as xr
from shapely.wkt import loads
from shapely.geometry import box

In [11]:
scene_path

'/dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE'

In [12]:
dataset = gdal.Open(scene_path+'/MTD_MSIL2A.xml')
metadata = dataset.GetMetadata()

/dss/dsshome1/06/di38laz/micromamba/envs/s2_dateline/lib/python3.13/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [13]:
footprint = metadata.get("FOOTPRINT", "N/A")
footprint

'POLYGON((180 70.27655361955614, 179.81353974733753 70.1729725023059, 179.5843586078059 70.04389269326435, 179.3585472875138 69.91458438853955, 179.13475573377443 69.78503878400527, 178.91353923 69.65517709587611, 178.69476259633996 69.52505546398446, 178.47855751966654 69.39473575765255, 178.26472185512287 69.26426455589153, 178.24780456916386 69.25384237584726, 177.9374906575181 69.24723897070908, 177.69674004388096 70.22784455372427, 180 70.27717192843326, -179.99884 70.27719672151753, 180 70.27655361955614))'

In [14]:
# Convert to a Shapely polygon
polygon = loads(footprint)
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:4326")

In [15]:
gdf

,geometry
0,"POLYGON ((180 70.27655, 179.81354 70.17297, 17..."


In [16]:
gdf.total_bounds

array([-179.99884   ,   69.24723897,  180.        ,   70.27719672])

### Plot the original geometry

In [17]:
gdf.explore()

### Plot the bounding box of the geometry

In [18]:
bbox = gdf.total_bounds

In [19]:
type(bbox)

numpy.ndarray

In [20]:
gdf_bbox = gpd.GeoDataFrame(
    {"geometry": [box(*bbox)]}, 
    crs=gdf.crs  # Use the CRS of the dataset
)


In [21]:
gdf_bbox.explore()

## Apply fix to one example

In [22]:
gdf['geometry'].apply(crosses_dateline)

0    True
Name: geometry, dtype: bool

In [23]:
gdf_bbox['geometry'].apply(crosses_dateline)

0    True
Name: geometry, dtype: bool

In [24]:
gdf_corr = gdf['geometry'].apply(correct_dateline_crossing)

In [25]:
print(gdf.total_bounds)
print(gdf_corr.total_bounds)

[-179.99884      69.24723897  180.           70.27719672]
[177.69674004  69.24723897 180.          70.27719672]


In [26]:
gdf_bbox_corr = gdf_bbox['geometry'].apply(correct_dateline_crossing)

In [27]:
print(gdf_bbox.total_bounds)
print(gdf_bbox_corr.total_bounds)

[-179.99884      69.24723897  180.           70.27719672]
[-180.           69.24723897 -179.99884      70.27719672]


### Plot the corrected geometry
This fix looks good.

In [28]:
gdf_corr.explore()

### Plot the corrected bbox geometry
This doesn't make sense if the bbox is spanning the whole globe before. Better to correct the original geometry!

In [29]:
gdf_bbox_corr.explore()

## Summary
**Bbest to apply correction on footprint, not on bbox.**

# Apply fix to all known cases
## Get instances where error occurs
### via camunda

https://ingestion.terrabyte.lrz.de/camunda/app/cockpit/default/#/process-definition/sentinel-data-ingestion:1:db72cd6b-952f-11ef-8d59-0242ac120003/runtime?searchQuery=%5B%5D&viewbox=%7B%22Definitions_1bkjt5n%22:%7B%22x%22:-191.70461095100868,%22y%22:84.95244956772335,%22width%22:1962.536023054755,%22height%22:469.7406340057637%7D%7D

camunda -> sentinel data ingestion workflow -> Extract Metadata -> Incidents -> Metadata Error

### globally
would be good to run a check against the STAC API whether everything is ok there

In [30]:
## why is antimeridian and the solution in place not solving this --> see comment: 

#stactools-sentinel2/src/stactools/sentinel2/stac.py at main · DLR-terrabyte/stactools-sentinel2
#https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056 
#https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056?.language=de 

### Make instance list

In [31]:
# manual for testing
scene_list = ['/dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE', 
              '/dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CT/2023/04/S2A_MSIL2A_20230409T002611_N0509_R102_T01WCT_20230409T040056.SAFE']

In [32]:
# extracted by Jonas M. via camunda scraping tool (got all metadata errors, this list does not only contain dateline errors)
scene_list = []
sen3_list = []

with open('camunda_sentinel_ingestion_errors_20250522.txt', 'r') as f:
    for line in f:
        filename = line.strip()
        if not filename:
            continue  # skip empty lines
        if filename.lower().endswith('.safe'):
            scene_list.append(filename)
        elif filename.lower().endswith('.sen3'):
            sen3_list.append(filename)

# print counts
print(f"SAFE files: {len(scene_list)}")
print(f"SEN3 files: {len(sen3_list)}")

SAFE files: 45
SEN3 files: 8


## Convenience functions to  facilitate looping through cases

In [33]:
#get footprint
#check if crossing --> defined above: crosses_dateline
#apply fix --> defined above: correct_dateline_crossing
#return
#visualize before and after

def get_footprint(scene_path):
    # get footprint from metadata
    dataset = gdal.Open(scene_path+'/MTD_MSIL2A.xml')
    metadata = dataset.GetMetadata()
    footprint = metadata.get("FOOTPRINT", "N/A")
    # Convert to a Shapely polygon and Create a GeoDataFrame
    polygon = loads(footprint)
    gdf = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:4326")
    #print(gdf)
    return(gdf)

def make_bbox(gdf):
    bbox = gdf.total_bounds
    gdf_bbox = gpd.GeoDataFrame(
        {"geometry": [box(*bbox)]}, 
        crs=gdf.crs
    )
    #print(gdf_bbox)
    return(gdf_bbox)

def check_and_correct(scene_path):
    print(f"Processing: {scene_path}")
    gdf = get_footprint(scene_path)
    gdf_bbox = make_bbox(gdf)

    # Default empty GeoDataFrames for no-correction case
    empty_gdf = gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

    # Check if correction is needed
    if gdf['geometry'].apply(crosses_dateline).any() or gdf_bbox['geometry'].apply(crosses_dateline).any():
        print("Correction needed. Applying fix.")
        # Correct dateline crossing
        corrected_geoms = gdf['geometry'].apply(correct_dateline_crossing)
        gdf_corr = gpd.GeoDataFrame(geometry=corrected_geoms, crs="EPSG:4326")
        gdf_corr_bbox = make_bbox(gdf_corr)
    else:
        print("No correction needed.")
        gdf_corr = empty_gdf
        gdf_corr_bbox = empty_gdf

    return gdf, gdf_bbox, gdf_corr, gdf_corr_bbox

    

## Apply fix to all cases

In [34]:
results = {} 
for scene_path in scene_list:
    gdf, gdf_bbox, gdf_corr, gdf_corr_bbox = check_and_correct(scene_path)
    results[scene_path] = {"gdf": gdf, "gdf_bbox": gdf_bbox, "gdf_corr": gdf_corr, "gdf_corr_bbox": gdf_corr_bbox}

Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/32/S/QD/2025/05/S2B_MSIL2A_20250514T100029_N0511_R122_T32SQD_20250514T135713.SAFE
No correction needed.
Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CM/2018/04/S2A_MSIL2A_20180405T235621_N0500_R116_T01WCM_20230730T095628.SAFE
Correction needed. Applying fix.
Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CM/2022/06/S2A_MSIL2A_20220603T235621_N0510_R116_T01WCM_20240615T221507.SAFE
Correction needed. Applying fix.
Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CM/2021/04/S2A_MSIL2A_20210409T235611_N0500_R116_T01WCM_20230510T190456.SAFE
Correction needed. Applying fix.
Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-0008/Sentinel-2-Col-1/L2A/tiles/01/W/CM/2018/06/S2B_MSIL2A_20180629T235619_N0500_R116_T01WCM_20230724T053921.SAFE
Correction needed. Applying fix.
Processing: /dss/dsstbyfs01/pn56su/pn56su-dss-

### Plot case by case to check if all corrections make sense
Change the index, use the layer control in the map to switch the layers on and off.

In [51]:
index = 12 # change index here to explore
scene_keys = list(results.keys())
key = scene_keys[index]
gdf = results[key]["gdf"]
gdf_bbox = results[key]["gdf_bbox"]
gdf_corr = results[key]["gdf_corr"]
gdf_corr_bbox = results[key]["gdf_corr_bbox"]

# Plot gdf_corr (blue) and gdf_corr_bbox (red)
m = gdf_corr.explore(color="blue", name="gdf_corr")
gdf_corr_bbox.explore(m=m, color="red", name="gdf_corr_bbox")
gdf.explore(m=m, color="green", name="gdf")
gdf_bbox.explore(m=m, color='yellow', name="gdf_bbox")
m.add_child(folium.LayerControl())


### Check tiles that are affected
It seems as if mostly the same tiles are affected.

In [65]:
# check if geometry was corrected
def is_corrected(gdf_corr):
    if gdf_corr is None:
        return False
    # If gdf_corr is a GeoSeries (from .apply), wrap it
    if isinstance(gdf_corr, gpd.GeoSeries):
        gdf_corr = gpd.GeoDataFrame(geometry=gdf_corr, crs="EPSG:4326")
    if not isinstance(gdf_corr, gpd.GeoDataFrame):
        return False
    # Check that it has at least one non-empty, non-null geometry
    return (
        gdf_corr.geometry.notnull().any() and
        not gdf_corr.geometry.is_empty.all()
    )

# Build correction summary
rows = []

for scene_path, geoms in results.items():
    filename = scene_path.split("/")[-1]
    parts = filename.split("_")

    N = next((p for p in parts if p.startswith("N") and len(p) == 5), None)
    R = next((p for p in parts if p.startswith("R") and len(p) == 4), None)
    T = next((p for p in parts if p.startswith("T") and len(p) == 6), None)

    gdf_corr = geoms.get("gdf_corr")
    correction_status = "Corrected" if is_corrected(gdf_corr) else "Not Corrected"

    rows.append((N, R, T, correction_status))

df = pd.DataFrame(rows, columns=["N", "R", "T", "Correction Status"])
summary = df.value_counts().reset_index(name="Count").sort_values("Count", ascending=False)

print(summary)

        N     R       T Correction Status  Count
0   N0500  R116  T01WCM         Corrected     28
1   N0509  R116  T01WCM         Corrected      4
2   N0510  R102  T01WCT         Corrected      3
3   N0509  R102  T01WCT         Corrected      2
4   N0510  R116  T01WCM         Corrected      1
5   N0511  R036  T37XDC     Not Corrected      1
6   N0511  R061  T48RUN     Not Corrected      1
7   N0511  R093  T35UPB     Not Corrected      1
8   N0511  R102  T01WCT         Corrected      1
9   N0511  R122  T32SQD     Not Corrected      1
10  N0511  R124  T22LFJ     Not Corrected      1
11  N0511  R132  T51SUA     Not Corrected      1


# Integrate fix
## Decide where to apply fix
I would suggest adding the code after the part where the stac tools function `create_item()` is checking if the area is too large. If this error triggers the correction should be applied to the footprint. And the bbox has to be extracted again.

Integrate here: https://github.com/DLR-terrabyte/stactools-sentinel2/blob/main/src/stactools/sentinel2/stac.py#L137

```
# sometimes, antimeridian and/or polar crossing scenes on some platforms end up
# with geometries that cover the inverse area that they should, so nearly the
# entire globe. This has been seen to have different behaviors on different
# architectures and dependent library versions. To prevent these errors from
# resulting in a wildly-incorrect geometry, we fail here if the geometry
# is unreasonably large. Typical areas will no greater than 3, whereas an
# incorrect globe-covering geometry will have an area for 61110.

if (ga := geometry.area) > 100:
    raise Exception(f"Area of geometry is {ga}, which is too large to be correct.")

bbox = [round(v, COORD_ROUNDING) for v in antimeridian.bbox(geometry)]

```

### Suggested fix
Needs testing when integrated in package.

- Make the function `crosses_dateline()` and `correct_dateline_crossing()` available to the package.
- Check if geom is crossing dateline and is currently too large --> Fix
- Check if geom is still too large --> If yes break.
- Check if geom is still crossing dateline --> If yes break.
- Make the bbox of the geom


```
if (ga := geometry.area) > 100 & (crosses_dateline(geometry)):
    raise Warning(f"Geometry is crossing the dateline and area of geometry is {ga}, which is too large to be correct. It will be corrected.")
    geometry = correct_dateline_crossing(geometry)
       
if (ga := geometry.area) > 100:
    raise Execption(f"Area of geometry is {ga}, which is too large to be correct.")

if (crosses_dateline(geometry)) :
    raise Exception(f"Geometry is still crossing the dateline."

# i don't know what this is actually doing? I guess it is creating the bbox from the geometry. 
# i guess antimeridian.bbox(geometry) makes a bbox and returns it as a list [xmin, ymin, xmax), ymax] (https://github.com/gadomski/antimeridian/blob/main/src/antimeridian/_implementation.py#L748)
# bbox = [round(v, COORD_ROUNDING) for v in antimeridian.bbox(geometry)]

# We also still have to do that
bbox = [round(v, COORD_ROUNDING) for v in geometry.total_bounds]
```



**Quick test on behavior of creating bbox**

In [72]:
gdf.geometry.total_bounds

array([-179.9997    ,   63.88152371,  180.        ,   64.88942462])

In [71]:
[round(v, 0) for v in gdf.geometry.total_bounds]

[np.float64(-180.0), np.float64(64.0), np.float64(180.0), np.float64(65.0)]